In [1230]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn import linear_model 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn. preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
from sklearn.datasets import load_wine
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_digits
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA


In [1231]:
df = pd.read_csv('bengaluru_house_prices.csv')
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [1232]:
df.groupby('area_type')['area_type'].count()


area_type
Built-up  Area          2418
Carpet  Area              87
Plot  Area              2025
Super built-up  Area    8790
Name: area_type, dtype: int64

In [1233]:
df2 = df.drop(['society','availability','balcony','area_type'],axis = 1)
df2.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


DATA CLEANING

In [1234]:
df2.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [1235]:

df3 = df2.dropna()#dropuje wiersze w ktorych nie ma wartosci   df['bath'] = df['bath'].fillna(mediano)
df3.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

In [1236]:
df3['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [1237]:
df3['BHK'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))

df3.head()

C:\Users\adria\AppData\Local\Temp\ipykernel_3044\1144535215.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['BHK'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))


,location,size,total_sqft,bath,price,BHK
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [1238]:
df3.BHK.unique()
df3[df3.BHK>20]
# df3.drop('BHK')

,location,size,total_sqft,bath,price,BHK
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43


In [1239]:
df3.total_sqft.unique() # trzeba usunac ten range

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [1240]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [1241]:
df3[~df3['total_sqft'].apply(is_float)].head()# jesli nie udalo sie negacja ~, to wyswietl je


,location,size,total_sqft,bath,price,BHK
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000,4
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005,2
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490,2
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800,2


In [1242]:
def range_split(x):
    tokens = x.split(' - ')
    if len(tokens) == 2:
        avg = (float(tokens[0]) + float(tokens[1]))/2
        return avg
    try:
        return(float(x))
    except:
        return None

In [1243]:
df4 = df3.copy()
df4['total_sqft'] = df4['total_sqft'].apply(range_split)
df4['total_sqft'].unique()
df4.head()

,location,size,total_sqft,bath,price,BHK
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3
4,Kothanur,2 BHK,1200.0,2.0,51.00,2


In [1244]:
df5 = df4.copy()
df5['total_m2'] = df5.total_sqft*0.0929
df5 =df5.drop('total_sqft',axis = 1)
df5['price_per_m2'] = (df5.price *10000/df5['total_m2']).round(2)
df5.head()

,location,size,bath,price,BHK,total_m2,price_per_m2
0,Electronic City Phase II,2 BHK,2.0,39.07,2,98.1024,3982.57
1,Chikka Tirupathi,4 Bedroom,5.0,120.00,4,241.5400,4968.12
2,Uttarahalli,3 BHK,2.0,62.00,3,133.7760,4634.61
3,Lingadheeranahalli,3 BHK,3.0,95.00,3,141.3009,6723.24
4,Kothanur,2 BHK,2.0,51.00,2,111.4800,4574.81


In [1245]:
len(df5.location.unique())

1304

In [1246]:
location_stat = df5.groupby('location')['location'].count().sort_values(ascending=False)

In [1247]:
less_than_10 = location_stat[location_stat<10]
less_than_10

location
Kamakshipalya            9
Kaverappa Layout         9
Yemlur                   9
Sector 1 HSR Layout      9
4th Block Koramangala    9
                        ..
 Banaswadi               1
Kanakadasa Layout        1
Kanakapur main road      1
Kanakapura  Rod          1
whitefiled               1
Name: location, Length: 1051, dtype: int64

In [1248]:
df5. location = df5.location.apply(lambda x: 'other' if x in less_than_10 else x)
len(df5.location.unique())


254

In [1249]:
df5.head(5)

,location,size,bath,price,BHK,total_m2,price_per_m2
0,Electronic City Phase II,2 BHK,2.0,39.07,2,98.1024,3982.57
1,Chikka Tirupathi,4 Bedroom,5.0,120.00,4,241.5400,4968.12
2,Uttarahalli,3 BHK,2.0,62.00,3,133.7760,4634.61
3,Lingadheeranahalli,3 BHK,3.0,95.00,3,141.3009,6723.24
4,Kothanur,2 BHK,2.0,51.00,2,111.4800,4574.81


OUTLIERS

In [1250]:
df6.shape

(12532, 7)

In [1251]:
df6 = df5[(df5.total_m2/df5.BHK)>25]
df6.shape

(12532, 7)

In [1252]:
df6.price_per_m2.describe()

count     12532.000000
mean       6811.954026
std        4512.503515
min         288.300000
25%        4532.320000
50%        5703.880000
75%        7473.735000
max      189957.580000
Name: price_per_m2, dtype: float64

In [1253]:
#delete outliners
def remove_outliners(df):
    df_out = pd.DataFrame()
    for key,subdf in df.groupby('location'):#po zmianie location funkcja jest bardiej uniwersalne
        m = np.mean(subdf.price_per_m2)
        std = np.std(subdf.price_per_m2)
        reduced_df = subdf[(subdf.price_per_m2 >(m-std)) & (subdf.price_per_m2 <(m+std))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

In [1255]:
df7 = remove_outliners(df6)
df7.shape

(10323, 7)

In [1256]:
df6.shape

(12532, 7)